# 課題4

この課題では与えられた問題において、LinUCBとトンプソンサンプリングを実装し、100回試行した後でregretの平均値について比較する。
ここではregretの計算式として、最も良い報酬の期待値を出す行動を$i^{\star}$としたとき、
\begin{align}
\text{regret} = (a_{i^{\star}} - a_{j})^{\top} \theta^{\star}
\end{align}
となるようにした。最初に実装に使う変数の設定等を行う。

In [3]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(seed = 42)
T = 10000
LinUCB_regret = []
thompson_regret = []
a = np.array([[np.cos(i * np.pi / 4), np.sin(i * np.pi / 4)] 
              for i in range(1, 9)])
theta_star = np.array([3, 1])

In [4]:
mu = [a[i].T @ theta_star for i in range(8)]
i_star = np.argmax(mu)

### LinUCB

まずLinUCBを実装する。実装は以下のようになる。

In [5]:
squared_sigma = 1
squared_sigma_0 = 1
alpha = 0.5

for loop in range(100):
    inv_A = (squared_sigma_0 / squared_sigma) * np.eye(2) 
    b = np.zeros(2)
    regret = 0
    for t in range(T):
        theta_hat = inv_A @ b
        mu_bar = []
        for i in range(8):
            ainv_Aa = a[i].T @ inv_A @ a[i]
            mu_bar.append(a[i].T @ theta_hat 
                + (alpha * np.sqrt(np.log(t + 1))) 
                          * np.sqrt(squared_sigma) * np.sqrt(ainv_Aa))
        i_obj = np.argmax(np.array(mu_bar))
        regret += (a[i_star] - a[i_obj]).T @ theta_star
        x = np.random.normal(a[i_obj].T @ theta_star, 1)
        aa_matrix = a[i_obj].reshape(-1, 1) @ a[i_obj].reshape(-1, 1).T
        ainv_Aa = a[i_obj].T @ inv_A @ a[i_obj]
        c = inv_A @ aa_matrix @ inv_A
        inv_A = inv_A - (c / (1 + ainv_Aa))
        b = b + a[i_obj] * x
    LinUCB_regret.append(regret)

In [6]:
print(theta_hat)
print(i_obj)
print(LinUCB_regret)

[2.99379924 0.97608258]
7
[603.3629277645632, 1717.042461036895, 542.1114112989612, 1721.2140339121486, 205.65700292627645, 85.04127162284418, 350.1802383930193, 909.5027610864819, 1709.3216816504746, 98.08081014213415, 77.4920651116763, 1443.5552978823584, 1303.2086859247606, 255.47201125292872, 88.12958337741286, 72.34487885406183, 1715.498305159611, 225.67215482792707, 1716.0130237853723, 96.87980001535745, 1343.0135929836392, 82.98239711979839, 80.9235226167526, 1719.3856067874026, 1714.8120136585958, 1721.2140339121481, 278.68817328135265, 1719.3856067874024, 137.3121240722121, 78.52150236319919, 78.34992948794537, 158.98918085723864, 1304.9244146772985, 75.43319060863051, 1721.2140339121481, 1715.2140339121493, 76.63420073540722, 99.45339314416468, 504.70852449363565, 384.54863731293926, 1719.3856067874024, 1716.5277424111337, 192.61746440698647, 72.0017331035542, 212.8630636869367, 878.1099755489221, 81.43824124251405, 101.68384052246428, 1718.3561695358796, 77.83521086218393, 6

### トンプソンサンプリング

次にトンプソンサンプリングを実装する。実装は以下のようになる。

In [7]:
squared_sigma = 1
squared_sigma_0 = 1

for loop in range(100):
    inv_A = (squared_sigma_0 / squared_sigma) * np.eye(2) 
    b = np.zeros(2)
    regret = 0
    for t in range(T):
        theta_tilde = np.random.multivariate_normal(inv_A @ b, 
                                                    squared_sigma * inv_A)
        mu_bar = []
        for i in range(8):
            mu_bar.append(a[i].T @ theta_tilde)
        i_obj = np.argmax(mu_bar)
        regret += (a[i_star] - a[i_obj]).T @ theta_star
        x = np.random.normal(a[i_obj].T @ theta_star, 1)
        aa_matrix = a[i_obj].reshape(-1, 1) @ a[i_obj].reshape(-1, 1).T
        ainv_Aa = a[i_obj].T @ inv_A @ a[i_obj]
        c = inv_A @ aa_matrix @ inv_A
        inv_A = inv_A - (c / (1 + ainv_Aa))
        b = b + a[i_obj] * x
    thompson_regret.append(regret)

In [9]:
print(theta_tilde)
print(i_obj)
print(thompson_regret)

[2.99300653 0.60713272]
7
[20.796897832170238, 11.063925136929054, 44.214069987373406, 20.01010126776666, 127.51522624004168, 14.519769259644784, 94.76205100926602, 26.292280474175435, 43.88311754568554, 22.318759132868053, 45.00591718566031, 46.870924236865754, 80.52864503813576, 15.904545570495019, 5.490332008121906, 41.20901935349008, 35.16738879314752, 58.672872702919264, 61.58961158223401, 121.18427379835362, 36.49329060095203, 41.57445968058414, 47.160246118211006, 13.992857325063591, 25.554257145050904, 41.255700547716046, 58.9621945842645, 35.944084089784404, 62.839394944289694, 35.09632098128204, 34.99581591789371, 110.89704395806109, 37.32171772569821, 50.785571075127415, 30.06688372975922, 76.17835661269156, 30.55930777893421, 20.82633508369309, 17.992857325063582, 42.206927312436996, 25.855772335215907, 40.90036148838867, 29.209019353490177, 23.500433275888547, 45.101371615165306, 20.110606331155, 30.92474810602826, 44.26075118159936, 25.642568899619484, 31.559307778934194,

### まとめ

以上のより、$T=10000$の時、LinUCBとトンプソンサンプリングを100回試行した時のregretの平均値はそれぞれ、

In [5]:
print(f"LinUCBのregretの平均値             :\
    {np.array(LinUCB_regret).mean()}")
print(f"トンプソンサンプリングのregretの平均値 :\
    {np.array(thompson_regret).mean()}")

LinUCBのregretの平均値             :    658.1098639975863
トンプソンサンプリングのregretの平均値 :    45.9782324275683


となっている。この結果を見るとLinUCBのregretの平均値はトンプソンサンプリングのregret平均値よりも大きいものになっていることがわかる。